<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(e)_Function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U -q "google-generativeai>=0.7.2"

In [3]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## **Set up a model with tools:**

This example uses 3 functions that control a simple hypothetical lighting system. Using these functions requires them to be called in a specific order. For example, you must turn the light system on before you can change color.

While you can pass these directly to the model and let it try to call them correctly, specifying the function_calling_config gives you precise control over the functions that are available to the model.




In [4]:
def enable_lights():
    """Turn on the lighting system."""  # telling to LLM regarding this function in tripple qutation.
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""   # telling to LLM regarding this function in tripple qutation.
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""   # telling to LLM regarding this function in tripple qutation.
    print("LIGHTBOT: Lights turned off.")

light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=light_controls,  # Functions calling via tools
    system_instruction=instruction
)

# Now your model is ready for function calling

In [5]:
chat=model.start_chat()

## **Creating a helper function:**

- The helper function is to designed to make things easier for developers.
- It takes simple, human friendly inputs ( Mode in str, & a list of allowed function names) & automatically converts those provided inputs into the detailed configuration format that Gemini API requires.
- This abstraction helps developers avoid manual errors and speed up the process, ensuring the configuration is always correct.


Developers after creation of this function only provide the `mode` and a list of allowed function names. The helper functions then automatically transforms these into the detailed structure the API requires.




In [6]:
# helper function
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

**Types of Modes:**

The selection of modes is very important, the developer is basically delegating power to the LLM to use the functions based on the LLM's judgement.

**1. `NONE`:** If you have provided the model with tools, but do not want to use those tools(functions) for the current conversational turn, then specify NONE as the mode. `NONE` tells the model not to make any function calls. The model will behave as `none` have been provided in the tools.

**2. `Auto`:** When developer write the `Auto` mode it means that the developer is instructing the LLM to use only one function in the allowed list based on your best judgements.  

**3. `Any`:** When developer write the `Any` mode it means that the developer is instructing the LLM to use any function (not be bound to a single function selection) in the allowed list.

In [7]:
# NONE mode:

tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hello light-bot, what can you do?", tool_config=tool_config
)
print(response.text)



I can turn lights on and off, and I can set the color of the lights.



In [ ]:
# AUTO Mode
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts)
chat.rewind();  # You are not actually calling the function, so remove this from the history.

In [ ]:
# Any Mode

available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place PURPLE!", tool_config=tool_config)
print(response.parts[0])

## **Automatic function calling**

In [ ]:
available_fns = ["enable_lights", "set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
auto_chat.send_message("It's awful dark in here- make it orange", tool_config=tool_config)